In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

from gams import *

In [2]:
# Load the gams_magic, run tutorial notebook for function definition

%load_ext gams_magic
%gams_cleanup -k

In [3]:
# Aux functions from the DataTransform notebook

def gt_from2dim(df, column_names=None):
    if column_names==None:
        return pd.DataFrame(df.stack()).reset_index()
    else:
        df = pd.DataFrame(df.stack()).reset_index()
        return df.rename(columns=dict(zip(df.columns, column_names)))

def gt_pivot(df, index=None, columns=None, values=None):
    df = df.pivot(index=index, columns=columns, values=values)
    df.index.names = [None]
    df.columns.names = [None]
    return df

def gt_pivot2d(df):
    return gt_pivot(df,index=df.columns[0], columns=df.columns[1], values=df.columns[2])

In [10]:
%%gams

** Define the structure to connect with the matlab code

** Indicate the option of the gams optimization process
*option optcr=0;
*option optca=0;
option iterlim = 1000000000;
option reslim = 5000000000;
*option sysout=on;

** Define the fixed sets structure to use in the parameters of the gams optimization process
set genLimitInfo 'Indicate the number of columns of information from generators'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set loadLimitInfo 'Indicate the number of columns of information from loads'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set storLimitInfo 'Indicate the number of columns of information from storage units'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set v2gLimitInfo 'Indicate the number of columns of information from electric vehicles'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set csLimitInfo 'Indicate the number of columns of information from charging stations'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;


* Define the several sets structure used in the gams optimization process, these sets depends the number of resources used in matlab code
set t /1*%period%/;

set gen /1*%gen%/;

set load /1*%Load%/;

set stor /1*%stor%/;

set v2g /1*%v2g%/;

set cs /1*%cs%/;

GamsExceptionExecution: GAMS return code not 0 (2), check C:\Users\camar\Documents\Jupyter\eso_project\gamsJupyter4.lst for more details

In [ ]:
%%gams

** Define the structure to connect with the matlab code
$onempty
#$include matglobs.gms

** Indicate the option of the gams optimization process
*option optcr=0;
*option optca=0;
option iterlim = 1000000000;
option reslim = 5000000000;
*option sysout=on;

** Define the fixed sets structure to use in the parameters of the gams optimization process
set genLimitInfo 'Indicate the number of columns of information from generators'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set loadLimitInfo 'Indicate the number of columns of information from loads'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set storLimitInfo 'Indicate the number of columns of information from storage units'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set v2gLimitInfo 'Indicate the number of columns of information from electric vehicles'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set csLimitInfo 'Indicate the number of columns of information from charging stations'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

* Define the several sets structure used in the gams optimization process, these sets depends the number of resources used in matlab code
set t /1*%period%/;

set gen /1*%gen%/;

set load /1*%Load%/;

set stor /1*%stor%/;

set v2g /1*%v2g%/;

set cs /1*%cs%/;

* Define the parameters with the data from the excel, such as limits, price, and voltage
parameter loadLimit(load,t,loadLimitInfo);
parameter genLimit (gen,t,genLimitInfo);
parameter genInfo(gen,genLimitInfo);
parameter pMaxImp(t),buyPrice(t);
parameter pMaxExp(t),sellPrice(t);
parameter storLimit(stor,t,storLimitInfo);
parameter storInfo(stor,storLimitInfo);
parameter v2gLimit(v2g,t,v2gLimitInfo);
parameter v2gInfo(v2g,v2gLimitInfo);
parameter csLimit (cs,t,csLimitInfo);
parameter csInfo (cs,csLimitInfo);
parameter EV_CS_Info(v2g,cs,t);
*parameter ev_x(t);

$if exist matdata $include matdata.gms
** Terminate the structure to connect with the matlab code

parameters  loadActPower(load,t), optCost;

loadActPower(load,t)=loadLimit(load,t,'1');

** Define the variables for the gams optimization process
variables fun, derCost, selfConso;

positive variables genActPower(gen,t), genExcActPower(gen,t), loadRedActPower(load,t), loadCutActPower(load,t), loadENS(load,t), pImp(t), pExp(t),storEnerState(stor,t),
                   storDchActPower(stor,t), storChActPower(stor,t), EminRelaxStor(stor,t), v2gDchActPower(v2g,t), v2gChActPower(v2g,t),v2gEnerState(v2g,t), EminRelaxEv(v2g,t),
                   csActPower(cs,t), csActPowerNet(cs,t);

binary variables genXo(gen,t), loadXo(load,t), v2gChXo(v2g,t), v2gDchXo(v2g,t), storChXo(stor,t), storDchXo(stor,t);
;


equations
objFun
optCostFun
optSelfFun

MaxImpEq(t)
MaxExpEq(t)

genActMaxEq1(gen,t)
genActMaxEq2(gen,t)

loadReactEq(load,t)
loadCutActPowerEq(load,t)
loadENSEq(load,t)

storDchRateIneq(stor,t)
storChRateIneq(stor,t)
storMaxEq(stor,t)
storRelaxEq(stor,t)
storBatBalanceIniEq(stor,t)
storBatBalanceEq(stor,t)
storBin(stor,t)

v2gDchRateIneq(v2g,t)
v2gChRateIneq(v2g,t)
v2gMaxEq(v2g,t)
v2gRelaxEq(v2g,t)
v2gRelaxReqEq1(v2g,t)
v2gRelaxReqEq2(v2g,t)
v2gRelaxReqEq3(v2g,t)

v2gBatBalanceZero(v2g,t)
v2gBatBalanceIniEq(v2g,t)
v2gBatBalanceEq(v2g,t)
v2gBatBalanceEq2(v2g,t)

v2gBin(v2g,t)

csActMaxEq(cs,t)
csActMinEq(cs,t)
csActPowerEq(cs,t)
csActPowerNetEq(cs,t)

actBalanceEq(t)
;


objFun.. fun =e= selfConso;

** Cost Minimization Objective Cost Function
optCostFun.. derCost=e=  sum((gen,t), genActPower(gen,t)*genLimit(gen,t,'3')+genExcActPower(gen,t)*genLimit(gen,t,'5'))
                       + sum((load,t), loadRedActPower(load,t)*loadLimit(load,t,'7')+loadCutActPower(load,t)*loadLimit(load,t,'8')+loadENS(load,t)*loadLimit(load,t,'10'))
                       + sum((stor,t), storDchActPower(stor,t)*storLimit(stor,t,'4')-storChActPower(stor,t)*storLimit(stor,t,'3') + EminRelaxStor(stor,t)*200)
                       + sum((v2g,t), v2gDchActPower(v2g,t)*v2gLimit(v2g,t,'7')-v2gChActPower(v2g,t)*v2gLimit(v2g,t,'6') + EminRelaxEv(v2g,t)*200)
                       + sum(t,PImp(t)*buyPrice(t)-PExp(t)*sellPrice(t))
                       ;

**  Self-Consuption Maximization Objective Cost Function
optSelfFun.. selfConso=e=sum(t,PImp(t))
                       + sum((gen,t), 10000*genExcActPower(gen,t))+ sum((load,t), 10000*loadENS(load,t))
                       + sum((stor,t),1000*EminRelaxStor(stor,t)) + sum((v2g,t), 1000*EminRelaxEv(v2g,t))
                       + sum((load,t),10*(loadRedActPower(load,t) + loadCutActPower(load,t)))
                       + sum(t,PExp(t)*100*(sum ((stor),(storLimit(stor,t,'1')-storChActPower(stor,t))) + sum((cs), csInfo(cs,'5')-csActPower(cs,t))))
                       + sum(t,PExp(t)*500*(sum ((stor),(storDchActPower(stor,t)))))
                       + 0.0001*derCost
                       ;

** Network ocntraints
* Max Import
MaxImpEq(t)..  PImp(t)=l=pMaxImp(t);
* Max Export
MaxExpEq(t)..  PExp(t)=l=pMaxExp(t);

** Generator constraints with the active generation power
*Maximum generation in generators with normal contract
genActMaxEq1(gen,t)$(genInfo(gen,'5')=1).. genActPower(gen,t)=e=genLimit(gen,t,'1')*genXo(gen,t);
*Generation in generators with Feed-in tariffs
genActMaxEq2(gen,t)$(genInfo(gen,'5')=2).. genActPower(gen,t)+genExcActPower(gen,t)=e=genLimit(gen,t,'1');

** Load constraints
*Load reduction constraint
loadReactEq(load,t) ..        loadRedActPower(load,t)=l=loadLimit(load,t,'3');
*Load curtailment constraint
loadCutActPowerEq(load,t) ..  loadCutActPower(load,t)=e=loadLimit(load,t,'4')*loadXo(load,t);
*Non-supplied energy constraint
loadENSEq(load,t) ..  loadENS(load,t)+loadRedActPower(load,t)+loadCutActPower(load,t)=l=loadActPower(load,t);

** Storage constraints
* Discharge Limit
storDchRateIneq(stor,t) ..    storDchActPower(stor,t)=l=storLimit(stor,t,'2')*storDchXo(stor,t);
* Charge Limit
storChRateIneq(stor,t) ..     storChActPower(stor,t)=l=storLimit(stor,t,'1')*storChXo(stor,t);
*Energy Limits
storMaxEq(stor,t) ..   storEnerState(stor,t) =l= storInfo(stor,'6');
storRelaxEq(stor,t) .. storEnerState(stor,t) =g= storInfo(stor,'7')*storInfo(stor,'6')/100 - EminRelaxStor(stor,t);
*Energy balance
storBatBalanceIniEq(stor,t)$(ord(t)=1) .. storEnerState(stor,t)=e= storInfo(stor,'6')*storInfo(stor,'10')/100 + storChActPower(stor,t)*storInfo(stor,'8')/100 - storDchActPower(stor,t)/(storInfo(stor,'9')/100);
storBatBalanceEq(stor,t)$(ord(t)>1) ..    storEnerState(stor,t)=e= storEnerState(stor,t-1) + storChActPower(stor,t)*storInfo(stor,'8')/100 - storDchActPower(stor,t)/(storInfo(stor,'9')/100);

* disable Charge and discharge in same period
storBin(stor,t).. storChXo(stor,t)+storDchXo(stor,t)=l=1;

** Electric Vehicles constraints
* Discharge Limit
v2gDchRateIneq(v2g,t) ..    v2gDchActPower(v2g,t)=l=v2gLimit(v2g,t,'5')*v2gLimit(v2g,t,'1')*v2gDchXo(v2g,t);
* Charge Limit
v2gChRateIneq(v2g,t) ..     v2gChActPower(v2g,t)=l=v2gLimit(v2g,t,'4')*v2gLimit(v2g,t,'1')*v2gChXo(v2g,t);

* Energy Limits
v2gMaxEq(v2g,t) ..   v2gEnerState(v2g,t)=l= v2gInfo(v2g,'5');
v2gRelaxEq(v2g,t)$(v2gLimit(v2g,t,'1')=1) .. v2gEnerState(v2g,t) =g= v2gInfo(v2g,'11')*v2gInfo(v2g,'5')/100 - EminRelaxEv(v2g,t);

* Energy Limits - If requirement
v2gRelaxReqEq1(v2g,t)$(v2gLimit(v2g,t,'1')=1).. v2gEnerState(v2g,t) =g= v2gLimit(v2g,t,'3')- EminRelaxEv(v2g,t);
v2gRelaxReqEq2(v2g,t)$(v2gLimit(v2g,t,'1')=1 and v2gLimit(v2g,t+1,'1')=0 and v2gLimit(v2g,t,'3')=0 and ord(t)<t.last).. v2gEnerState(v2g,t) =g= v2gInfo(v2g,'5')- EminRelaxEv(v2g,t);
v2gRelaxReqEq3(v2g,t)$(v2gLimit(v2g,t,'1')=1 and v2gLimit(v2g,t,'3')=0 and ord(t)=t.last).. v2gEnerState(v2g,t) =g= v2gInfo(v2g,'5')- EminRelaxEv(v2g,t);

*Energy balance
v2gBatBalanceZero(v2g,t)$(v2gLimit(v2g,t,'1')=0).. v2gEnerState(v2g,t)=e=0;
v2gBatBalanceIniEq(v2g,t)$(v2gLimit(v2g,t,'1')=1 and ord(t)=1).. v2gEnerState(v2g,t)=e=v2gLimit(v2g,t,'2')+v2gChActPower(v2g,t)*v2gInfo(v2g,'8') - v2gDchActPower(v2g,t)/v2gInfo(v2g,'9');
v2gBatBalanceEq(v2g,t)$(v2gLimit(v2g,t-1,'1')=1 and v2gLimit(v2g,t,'1')=1 and ord(t)>1) ..    v2gEnerState(v2g,t)=e= v2gEnerState(v2g,t-1) + v2gLimit(v2g,t,'2') + v2gChActPower(v2g,t)*v2gInfo(v2g,'8') - v2gDchActPower(v2g,t)/v2gInfo(v2g,'9');
v2gBatBalanceEq2(v2g,t)$(v2gLimit(v2g,t-1,'1')=0 and v2gLimit(v2g,t,'1')=1 and ord(t)>1) ..    v2gEnerState(v2g,t)=e= v2gLimit(v2g,t,'2') + v2gChActPower(v2g,t)*v2gInfo(v2g,'8') + v2gDchActPower(v2g,t)/v2gInfo(v2g,'9');

* disable charge and discharge in same period
v2gBin(v2g,t).. v2gChXo(v2g,t)+v2gDchXo(v2g,t)=l=1;

** Charging Station constraints
* Charge Limits
csActMaxEq(cs,t) .. csActPower(cs,t) =l= csInfo(cs,'5');
csActMinEq(cs,t) .. csActPower(cs,t) =g= - csInfo(cs,'6');
csActPowerEq(cs,t) ..   csActPower(cs,t) =e= sum(v2g$EV_CS_Info(v2g,cs,t), v2gChActPower(v2g,t) - v2gDchActPower(v2g,t));
csActPowerNetEq(cs,t) ..   csActPowerNet(cs,t) =e= sum(v2g$EV_CS_Info(v2g,cs,t), v2gChActPower(v2g,t)/(csInfo(cs,'7')/100) - v2gDchActPower(v2g,t)*csInfo(cs,'8')/100);

* Active and reactive power balance
actBalanceEq(t).. 0=e=sum(gen,genActPower(gen,t) - genExcActPower(gen,t))
                    + PImp(t)-PExp(t)
                    - sum(load, loadActPower(load,t) - loadRedActPower(load,t) - loadCutActPower(load,t) - loadENS(load,t))
                    - sum(stor, storChActPower(stor,t) - storDchActPower(stor,t))
*                   - sum(v2g, v2gChActPower(v2g,t) - v2gDchActPower(v2g,t))
                    - sum(cs, csActPowerNet(cs,t))


Model optProblem  /all/;
option minlp=dicopt;
optProblem.optfile=1;
*option mip=cplex;
*option nlp=conopt;

*optProblem.WORKFACTOR=10.0;

$if exist matglobs.gms $include matglobs.gms
solve optProblem using minlp minimizing fun;

* Specify a variable to contain the status of the optimization problem
scalar statusOptPro;
parameter LoadTot(load,t);
statusOptPro=optProblem.modelstat;
LoadTot(load,t)=loadActPower(load,t)-loadRedActPower.l(load,t)-loadCutActPower.l(load,t)-loadENS.l(load,t);

* Define the output results
* Pass the general results, objective function, LMP, problem status
$libinclude matout fun.l
$libinclude matout derCost.l
$libinclude matout selfConso.l
$libinclude matout statusOptPro

*  Pass import, export energy
$libinclude matout PImp.l t
$libinclude matout PExp.l t

* Pass the generator results, binary decision, active power, excess power, reactive power
$libinclude matout genActPower.l gen t
$libinclude matout genExcActPower.l gen t
$libinclude matout genXo.l gen t

* Pass the load results, binary decision, active power, excess power, reactive power
$libinclude matout loadRedActPower.l load t
$libinclude matout loadCutActPower.l load t
$libinclude matout loadENS.l load t
$libinclude matout loadXo.l load t
$libinclude matout LoadTot load t

* Pass the storage results, binary decision, active power, excess power, reactive power
$libinclude matout storEnerState.l stor t
$libinclude matout storChActPower.l stor t
$libinclude matout storDchActPower.l stor t

* Pass the eletric vehicles results, binary decision, active power, excess power, reactive power
$libinclude matout v2gEnerState.l v2g t
$libinclude matout v2gDchActPower.l v2g t
$libinclude matout v2gChActPower.l v2g t
$libinclude matout EminRelaxEv.l v2g t

$libinclude matout csActPower.l cs t


In [6]:
period = np.array([1, 2, 3])
gen = np.array([2, 3, 4])
Load = np.array([3, 4, 5])
stor = np.array([4, 5, 6])
v2g = np.array([5, 6, 7])
cs = np.array([6, 7, 8])

df = pd.DataFrame({'period': period,
                   'gen': gen,
                   'Load': Load,
                   'stor': stor,
                   'v2g': v2g,
                   'cs': cs},
                  index=['i1', 'i2', 'i3'])

gams_df = gt_from2dim(df)
i = df.index.values.tolist()
j = df.columns.values.tolist()

%gams_reset
%gams set i,j;
%gams parameter gams_df(i, j)

%gams_push i
%gams_push j
%gams_push gams_df
%gams display gams_df
%gams_lst -e

E x e c u t i o n


----     22 PARAMETER gams_df  

        period         gen        Load        stor         v2g          cs

i1       1.000       2.000       3.000       4.000       5.000       6.000
i2       2.000       3.000       4.000       5.000       6.000       7.000
i3       3.000       4.000       5.000       6.000       7.000       8.000



